In [1]:
#Implementing SCD Type 3

In [5]:
import pandas as pd
import sqlalchemy as sal
engine=sal.create_engine('mssql://DESKTOP-N6DEIMB\SQLEXPRESS/master?driver=ODBC+Driver+17+for+SQL+Server')
conn=engine.connect()

In [62]:
#implementing SCD Type1
def extract():
    df_products=pd.read_csv('product2.txt')
    df_products_db=pd.read_sql_query('select * from product_scd3',con=conn)
    return df_products,df_products_db

def transform(df_products,df_products_db):
    df_merge=pd.merge(df_products,df_products_db,how='left',on='product_id')
    df_insert=df_merge[df_merge['product_name_y'].isna()]
    df_insert_final=df_insert.iloc[:,0:3]
    df_insert_final.columns=[df_products_db.columns[0],df_products_db.columns[1],df_products_db.columns[3]]
    
    df_update=df_merge[df_merge['product_name_y'].notna()]
    df_update_final=df_update.iloc[:,0:3]
    df_update_final.columns=[df_products_db.columns[0],df_products_db.columns[1],df_products_db.columns[3]]
    return df_insert_final,df_update_final

def load_staging(df_update_final):
    df_update_final.to_sql('products_stg',con=conn , index=False , if_exists = 'replace')
    conn.commit()

def update():
    q=sal.text('update product_scd3 set old_product_name=product_name,old_product_price=product_price where product_id in (select a.product_id from product_scd3 a join products_stg b on a.product_id=b.product_id)')
    p = conn.execute(q)
    query = sal.text("update product_scd3 set product_price = products_stg.product_price,product_name=products_stg.product_name from products_stg where product_scd3.product_id=products_stg.product_id")
    p = conn.execute(query)
    conn.commit()
    
def load(df_insert_final):
    df_insert_final.to_sql('product_scd3',index=False,con=conn,if_exists='append')
    conn.commit()

In [63]:
df_products,df_products_db=extract()

In [64]:
df_insert_final,df_update_final=transform(df_products,df_products_db)

In [65]:
load(df_insert_final)

AttributeError: 'Connection' object has no attribute 'commit'

In [66]:
load_staging(df_update_final)

AttributeError: 'Connection' object has no attribute 'commit'

In [67]:
update()

AttributeError: 'Connection' object has no attribute 'commit'